<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Section-2---Adding-ATC-lvl2" data-toc-modified-id="Section-2---Adding-ATC-lvl2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Section 2 - Adding ATC lvl2</a></span></li></ul></div>

## Data cleaning

In [1]:
import numpy as np
import pandas as pd
import copy
import nachopy # Library with custom functions
import importlib # Used to rei
import datetime

First we read the data and remove any duplicates

In [3]:
# Hospital patient data
prescriptions = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/prescriptions.csv')
# Emergency Department (ED) patient data
medrecon = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/ed/medrecon.csv')

C:\Users\nacho\AppData\Local\Temp\ipykernel_19224\601855088.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/prescriptions.csv')


A warning given because the GSN column is both integers and lists of integers (some drugs have more than one gsn code), so we have mixed types

We can convert the NDC columns, which are only composed of integers. By using panda's dtype 'Int64' we can save a bit of memory by avoiding the float (numpy's int types cannot be used with null values)

In [7]:
prescriptions['ndc'] = prescriptions.ndc.astype('Int64')
medrecon['ndc'] = medrecon.ndc.astype('Int64')

In [8]:
display(prescriptions.head())
display(prescriptions.info(show_counts = True))

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,APAP500,004490,904198861,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,FURO40,008209,51079007320,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,RALT400,063231,6022761,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14924364 entries, 0 to 16219411
Data columns (total 20 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   subject_id         14924364 non-null  int64  
 1   hadm_id            14924364 non-null  int64  
 2   pharmacy_id        14924364 non-null  int64  
 3   poe_id             14821476 non-null  object 
 4   poe_seq            14821476 non-null  float64
 5   starttime          14916988 non-null  object 
 6   stoptime           14911010 non-null  object 
 7   drug_type          14924364 non-null  object 
 8   drug               14924363 non-null  object 
 9   formulary_drug_cd  14904840 non-null  object 
 10  gsn                13294245 non-null  object 
 11  ndc                14898848 non-null  Int64  
 12  prod_strength      14917950 non-null  object 
 13  form_rx            20208 non-null     object 
 14  dose_val_rx        14917996 non-null  object 
 15  dose_unit_rx 

None

In [9]:
display(medrecon.head())
display(medrecon.info(show_counts = True))

,subject_id,stay_id,charttime,name,gsn,ndc,etc_rn,etccode,etcdescription
0,10000032,32952584,2180-07-22 17:26:00,albuterol sulfate,28090,21695042308,1,5970.0,Asthma/COPD Therapy - Beta 2-Adrenergic Agents...
1,10000032,32952584,2180-07-22 17:26:00,calcium carbonate,1340,10135021101,1,733.0,Minerals and Electrolytes - Calcium Replacement
2,10000032,32952584,2180-07-22 17:26:00,cholecalciferol (vitamin D3),65241,37205024678,1,670.0,Vitamins - D Derivatives
3,10000032,32952584,2180-07-22 17:26:00,emtricitabine-tenofovir [Truvada],57883,35356007003,1,5849.0,Antiretroviral - Nucleoside and Nucleotide Ana...
4,10000032,32952584,2180-07-22 17:26:00,fluticasone [Flovent HFA],21251,49999061401,1,371.0,Asthma Therapy - Inhaled Corticosteroids (Gluc...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143791 entries, 0 to 3143790
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   subject_id      3143791 non-null  int64  
 1   stay_id         3143791 non-null  int64  
 2   charttime       3143791 non-null  object 
 3   name            3143791 non-null  object 
 4   gsn             3143791 non-null  int64  
 5   ndc             3143791 non-null  Int64  
 6   etc_rn          3143791 non-null  int64  
 7   etccode         3131412 non-null  float64
 8   etcdescription  3131412 non-null  object 
dtypes: Int64(1), float64(1), int64(4), object(3)
memory usage: 218.9+ MB


None

We have large datasets, although most of the columns will not be needed on the prescriptions table so we will be able to slim it down. Let's We can also check whether we have other NDC numbers with fewer digits, as these could be fixable mistakes.

Prescriptions and medrecon are similar,thankfully, although instead of hadm_id we have stay_id. And no null values on our drugs column, yay! We lack the drug type column.

In [11]:
prescriptions.drop(columns = ['prod_strength','form_rx','dose_val_rx','dose_unit_rx','form_val_disp',\
                              'form_unit_disp','doses_per_24_hrs','pharmacy_id','poe_id','poe_seq','stoptime'],inplace = True)


Medications are duplicated to include the different etc codes for one drug, which could be an interesting feature. However, there are no etc codes for the prescriptions table, so we will use ATCs instead, which is a more well-known standard. More importantly, they have a very customisable granularity, as we can select the level. At level 0 we have only 14 features, which simplifies our model greatly. We will start with that and increase granularity if necessary.

In [13]:
# We can define our duplicate with rows with a subset of relevant columns (some drugs may have different codes, duplicates with missing values...)
pres_dupl = prescriptions[['subject_id','hadm_id','starttime','stoptime','drug']].duplicated()
prescriptions = prescriptions[~pres_dupl]

# We will do the same for medrecon and rename some columns for the merging of the two tables
medrecon.drop(columns =  ['etccode','etc_rn','etcdescription'], inplace = True)
medrecon = medrecon[~dupl_rows]
medrecon.rename(columns = {'charttime':'starttime','name':'drug'},inplace = True)

In [15]:
nachopy.display_sbs([medrecon.stay_id.describe(),prescriptions.hadm_id.describe()],\
                    titles = ['Stay_id from medrecon','Hadm_id from prescriptions'])

,stay_id
count,2704564.000000
mean,35001695.562961
std,2885678.657628
min,30000012.000000
25%,32502217.000000
50%,35000352.000000
75%,37497977.000000
max,39999964.000000
,hadm_id
count,14924364.000000


Lovely, comparable, no overlap. We will combine them. We do not need to label the new stay_id as hadm_id as we can see that hadm_id have a range from 20 to 30 million and the stay_id from 30 to 40 million. 

In [16]:
prescriptions = pd.concat([prescriptions,medrecon.rename(columns = {'stay_id':'hadm_id'})],axis = 0)
prescriptions.reset_index(inplace = True,drop = True)

Let's convert the starttime column to datetime, which will be useful when merging tables

In [19]:
prescriptions['starttime'] = pd.to_datetime(prescriptions.starttime)

We need the ATC categories for each prescription, and we can get that using [Fabricio Kury's R script to map NDCs to RxNorm](https://github.com/fabkury/ndc_map) and to their ATC categories.

In [ ]:
# Exporting NDC list
pd.Series(prescriptions.ndc.unique()).to_csv('G:/My Drive/Capstone-Data/ndc_map/unique_ndcs.csv')

In [22]:
# Importing mapped_NDC list
mapped_NDC = pd.read_csv('G:/My Drive/Capstone-Data/ndc_map/output/atc4/ndc_map 2022_10_12 17_12 (atc4).csv',usecols = ['ndc','atc4','atc4_name'])

mapped_NDC.dropna(inplace = True) # Dropping any null values
mapped_NDC.ndc = mapped_NDC.ndc.astype('int64') # Converting to a more compact data type 
mapped_NDC.drop_duplicates(inplace = True) # Dropping any duplicate values
len(mapped_NDC)

21710

In [23]:
mapped_NDC.head()

,ndc,atc4,atc4_name
2,2050301,J01GB,Other aminoglycosides
3,2050301,S01AA,Antibiotics
4,2060440,J04AB,Antibiotics
5,2100902,R05DA,Opium alkaloids and derivatives
8,2143301,A10BJ,Glucagon-like peptide-1 (GLP-1) analogues


There is more than one atc1 per ndc code. We were going to use dummy variables in the future, so we will implement those now before the merge. That way we will have just one category per ndc. Now we have several levels of categories here. For example in atc4 J01GB, J is level 1, J01, level 2, J01G level 3 and J01GB level 4. Let's see how many features we would have for each of these.

In [25]:
mapped_NDC['atc3'] = mapped_NDC.atc4.str[:-1] # All but last element of string
mapped_NDC['atc2'] = mapped_NDC.atc4.str[:3] # First three
mapped_NDC['atc1'] = mapped_NDC.atc4.str[0] # Only first

# Printing unique values to know number of columns if added to prescriptions table as dummies
print('Number of categories')
print(f'ATC4: {mapped_NDC.atc4.nunique()}\nATC3: {mapped_NDC.atc3.nunique()}\nATC2: {mapped_NDC.atc2.nunique()}\nATC1: {mapped_NDC.atc1.nunique()}')

Number of categories
ATC4: 560
ATC3: 196
ATC2: 86
ATC1: 14


A lot of categories. Level 2 and 3 are feasible, but we will first try taking just the first letter (lvl 1 of classification) of the atc_4 instead. This will allow us to play around with the filling mechanisms without being hindered by excessively heavy dataframes. Also, the last two digits of the ndc correspond with a packaging size, so we can join to prescriptions without takinng these two into account (we might get more hits/less missing values)

In [26]:
mapped_NDC.ndc = mapped_NDC.ndc.floordiv(100) # Removing two last digits
atc1 = pd.get_dummies(mapped_NDC[['ndc','atc1']],prefix = '',prefix_sep = '') # Adding categories as dummy
atc1 = atc1.groupby(by = 'ndc', as_index=False).any()*1 # Category will be 1 when there is at least one row for ndc with that category
atc1.drop_duplicates(inplace = True)
display(atc1)

,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
0,20503,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,20604,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,21009,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,21433,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,21434,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10157,820280004,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10158,861690020,0,0,1,1,0,0,0,0,0,1,0,1,1,0
10159,876510760,0,0,0,1,0,0,1,0,0,1,0,0,1,0
10160,894110545,0,0,0,1,0,0,1,0,0,1,0,0,1,0


We can do the merging now. Let's see how missing values are after the merge on the categories.

In [27]:
# Filling NaNs for merging
prescriptions['ndc'] = prescriptions.ndc.fillna(0) 
# Only interested on first 9 digits
prescriptions['ndc'] = prescriptions.ndc.floordiv(100)
# Left join as we want to keep rows on prescriptions where there is no match (we will try to fill those rows)
cat_pres = pd.merge(prescriptions[['drug_type','drug','formulary_drug_cd','gsn','ndc']],atc1,on = 'ndc',how = 'left')

cat_pres

,join_on_index,drug_type,drug,formulary_drug_cd,gsn,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
0,0,MAIN,Acetaminophen,APAP500,004490,9041988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,MAIN,Furosemide,FURO40,008209,510790073,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,MAIN,Raltegravir,RALT400,063231,60227,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,MAIN,Heparin,HEPA5I,006549,633230262,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17628923,17628923,NaN,fluticasone,NaN,18368,122800270,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17628924,17628924,NaN,glipizide,NaN,1776,118451101,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17628925,17628925,NaN,ibuprofen,NaN,8350,105440071,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
17628926,17628926,NaN,metronidazole,NaN,9592,105440938,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
display(cat_pres.sample(10))
print('Missing categories %:',sum(cat_pres.A.isna())/len(cat_pres)*100)
cat = list('ABCDGHJLMNPRSV')
# Labelling null values in cat as -1 for the filling
cat_pres[cat] = cat_pres[cat].fillna(-1).astype('int8')
# Labelling null ndc values as NaN again
cat_pres['ndc'] = cat_pres.ndc.where(cat_pres.ndc>0)
cat_pres['ndc'] = cat_pres.ndc.astype('Int64')

print(cat_pres.info(show_counts = True))

,join_on_index,drug_type,drug,formulary_drug_cd,gsn,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
5301811,5301811,MAIN,Potassium Chloride,KCL20P,001262,764390343,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17020639,17020639,NaN,rosuvastatin,NaN,51786,136680182,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13633020,13633020,MAIN,Propofol,PROP200IG,016796,633230269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4786179,4786179,MAIN,Heparin,HEPA5I,006549,633230262,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15344592,15344592,NaN,albuterol sulfate,NaN,5039,122800241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12557776,12557776,BASE,0.9% Sodium Chloride,NS1000,001210,3380049,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12047408,12047408,BASE,SW,SW50,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444938,1444938,MAIN,Insulin,INRHI,001723,28215,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2210455,2210455,MAIN,Simvastatin,SIMV10,016577,9045800,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9206127,9206127,BASE,0.9% Sodium Chloride,NS500,001210,3380049,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Missing categories %: 17.792244656056226
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17628928 entries, 0 to 17628927
Data columns (total 20 columns):
 #   Column             Non-Null Count     Dtype 
---  ------             --------------     ----- 
 0   join_on_index      17628928 non-null  int64 
 1   drug_type          14924364 non-null  object
 2   drug               17628927 non-null  object
 3   formulary_drug_cd  14904840 non-null  object
 4   gsn                15998809 non-null  object
 5   ndc                15844153 non-null  Int64 
 6   A                  17628928 non-null  int8  
 7   B                  17628928 non-null  int8  
 8   C                  17628928 non-null  int8  
 9   D                  17628928 non-null  int8  
 10  G                  17628928 non-null  int8  
 11  H                  17628928 non-null  int8  
 12  J                  17628928 non-null  int8  
 13  L                  17628928 non-null  int8  
 14  M                  17628928 non-null  i

Note that the new category dummy columns are floats after the join, not int8s. We have filled the NaNs as -1 and changed the data type of the columns. This is useful to decrease the size of the dataframe and it will also be useful to fill the missing values (which is 19.26% of the rows), as we will see below. We will fill these missing values by matching with other columns and we there are a few candidates column for this: drug, pharmacy_id, gsn and formulary_drug_cd. The goal will be to find the column with values shared by the largest amount of rows, so matching ATR1s is more likely. 

In [29]:
nndrug = cat_pres[cat_pres.drug.isna()].index.values[0]
display(cat_pres.loc[nndrug])

display(cat_pres[cat_pres.ndc == cat_pres.ndc.loc[nndrug]].head(1))

cat_pres.loc[nndrug,['drug', 'formulary_drug_cd']] = ('Simvastatin','SIMV40')

print('Value counts of drugs:\n',cat_pres.drug.value_counts())
cat_pres.drug = cat_pres.drug.str.lower()
drugs_vc = cat_pres.drug.value_counts()

clin_drug = cat_pres.drug.str.contains('clindamyci')
display(cat_pres[clin_drug].head(1))
cat_pres.drug[clin_drug] = 'clindamycin'

print('\n\nValue counts of drugs after making all letters lowercase:\n',drugs_vc)
print('\nDescription:\n',drugs_vc.describe())

print('\nNumber of drugs used only once:', sum(drugs_vc>1))

cat_pres['drug'] = cat_pres.drug.str.replace('[^a-zA-Z]','', regex = True)
drugs_vc = cat_pres.drug.value_counts()

print('\n\nValue counts of drugs after making all letters lowercase:\n',drugs_vc)
print('\nDescription:\n',drugs_vc.describe())

print('\nNumber of drugs used only once:', sum(drugs_vc>1))



join_on_index         14845444
drug_type                 MAIN
drug                       NaN
formulary_drug_cd      SIMVIND
gsn                     016579
ndc                  510790456
A                            0
B                            0
C                            1
D                            0
G                            0
H                            0
J                            0
L                            0
M                            0
N                            0
P                            0
R                            0
S                            0
V                            0
Name: 14845444, dtype: object

,join_on_index,drug_type,drug,formulary_drug_cd,gsn,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
333,333,MAIN,Simvastatin,SIMV40,016579,510790456,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Value counts of drugs:
 Insulin                        557829
Sodium Chloride 0.9%  Flush    530628
0.9% Sodium Chloride           508792
Acetaminophen                  411170
Furosemide                     333397
                                ...  
*GENERIC VIAGRA                     1
Nisoldipine                         1
lactase [Lac-Dose]                  1
*citracal petites +D                1
*Atrovastatin                       1
Name: drug, Length: 25853, dtype: int64


,join_on_index,drug_type,drug,formulary_drug_cd,gsn,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
331,331,MAIN,clindamycin,CLIN9000I,009344,4094197,0,0,0,1,1,0,1,0,0,0,0,0,0,0


C:\Users\nacho\AppData\Local\Temp\ipykernel_19224\2913060166.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_pres.drug[clin_drug] = 'clindamycin'




Value counts of drugs after making all letters lowercase:
 insulin                        557853
sodium chloride 0.9%  flush    530628
0.9% sodium chloride           508792
acetaminophen                  441995
furosemide                     359143
                                ...  
*rhizotriptn                        1
*zelgenz                            1
*zeljanz                            1
*womens vitamin                     1
*atrovastatin                       1
Name: drug, Length: 22976, dtype: int64

Description:
 count     22976.000000
mean        767.275766
std       10722.416656
min           1.000000
25%           1.000000
50%           3.000000
75%          17.000000
max      557853.000000
Name: drug, dtype: float64

Number of drugs used only once: 13943


Value counts of drugs after making all letters lowercase:
 sodiumchloride         718560
insulin                557909
sodiumchlorideflush    530637
dextrose               452697
acetaminophen          441999
     

In [30]:
empty_drug = cat_pres.drug ==u''
print(f'There are {sum(empty_drug)} empty drugs')
display(cat_pres[empty_drug].sample())

fdrugs_empty = cat_pres.formulary_drug_cd[empty_drug].drop_duplicates().values
empty_drug = cat_pres.formulary_drug_cd.isin(fdrugs_empty)

cat_pres.drug.where(~empty_drug,cat_pres.formulary_drug_cd,inplace = True)

print(f'{sum(drugs_vc == 1)/len(cat_pres)*100}% of values in drugs column appears only once')

There are 112 empty drugs


,join_on_index,drug_type,drug,formulary_drug_cd,gsn,ndc,A,B,C,D,G,H,J,L,M,N,P,R,S,V
17435571,17435571,NaN,,NaN,0,<NA>,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


0.04262879739482741% of values in drugs column appears only once


There was just one missing value for the drug column, which was easily filled by checking elsewhere in the table. After this, we analysed the number of rows for each drug. Interestingly, the formulary_drug_code was different than the original for that row. 

The input of these fields does not seem consistent. For example "Clindamycin" is misspelt (fixed for this particular case), abbreviations may not be consistent and upper and lower case seem to be used with little care. This last issue could be fixed with str.lower(), though. 

This means too many groups so matching categories will be harder. Let us look at the other columns with drug identifiers.

In [31]:
print('gsn value counts:\n',cat_pres.gsn.value_counts())

fd_vc = cat_pres.formulary_drug_cd.value_counts()
print('\n\nformulary_drug_cd value counts:\n',fd_vc)
print('\nDescription:\n',fd_vc.describe())

fd_vc_5 = fd_vc[fd_vc<=5].value_counts()
print('\nNumber of formulary_drug_cd present on only 5 rows or less:\n',fd_vc_5)
print('\n% of rows with drugs used 5 times or less:', round(fd_vc_5.sum()/fd_vc.sum()*100,5))

gsn value counts:
 001210     757416
006549     320343
027413     252990
008205     247213
004489     242690
            ...  
065800          1
070413          1
004773          1
11474           1
73621           1
Name: gsn, Length: 15601, dtype: int64


formulary_drug_cd value counts:
 NACLFLUSH    530435
HEPA5I       320637
ACET325      243655
DOCU100      243283
SENN187      206732
              ...  
8ADDE5            1
HEPAR10I          1
ANDE100I          1
TMC125IND         1
OXACDESEN         1
Name: formulary_drug_cd, Length: 3789, dtype: int64

Description:
 count      3789.000000
mean       3933.713381
std       18224.888227
min           1.000000
25%          10.000000
50%          97.000000
75%        1118.000000
max      530435.000000
Name: formulary_drug_cd, dtype: float64

Number of formulary_drug_cd present on only 5 rows or less:
 1    288
2    151
4    106
3     99
5     81
Name: formulary_drug_cd, dtype: int64

% of rows with drugs used 5 times or less: 0.00486


From these candidates, formulary_drug_cd, gsn, and drugs seems the best to find the right ATR1s, but they have many null values. Additionally, the gsn column has 1.8 M missing values and sometimes several codes in one field. We will have to work around this
The pharmacy_id has many unique values, so this variable is not specific to the same drug, just an inventory code specific per item.  

We want to decrease the number of unique values as this will increase our chances of filling missing ATR1 fields. All our columns have many null values, but we can recursively fill categories based on all the identifying columns.

In [32]:
len(cat_pres)

17628928

In [33]:
cat_pres[cat].sum(axis = 1).value_counts()

 1     9430898
-14    3136582
 2     2662073
 3     1593912
 4      352304
 5      310606
 6       87288
 8       34041
 7       21224
dtype: int64

In [34]:
cat_pres.gsn.value_counts()

001210     757416
006549     320343
027413     252990
008205     247213
004489     242690
            ...  
065800          1
070413          1
004773          1
11474           1
73621           1
Name: gsn, Length: 15601, dtype: int64

In [35]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 17.792244656056226
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 16.88708994716776
Loop #1, missing values % after filling with drug column: 10.463635263704997
Loop #1, missing values % after filling with formulary_drug_cd column: 9.781970925723476
Loop #1, missing values % after filling with ndc column: 9.62126421668362

Loop #1, missing values % after re-imploding df: 9.593056367352569
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 9.571158974746517
Loop #2, missing values % after filling with drug column: 8.993817393261496
Loop #2, missing values % after filling with formulary_drug_cd column: 8.979206663985865
Loop #2, missing values % after filling with ndc column: 8.979099190561072

Loop #2, missing values % after re-imploding df: 8.982247814501255
Dataframe exploded, starting for loop #3

Loop #3, missing valu

In [36]:
importlib.reload(nachopy)

<module 'nachopy' from 'C:\\Users\\nacho\\Capstone-project-Brainstation\\nachopy.py'>

In [37]:
nachopy.display_sbs([cat_pres.drug[cat_pres.A<0].value_counts(normalize = True).iloc[0:50]*100],\
                    titles = ['Drugs value counts'],max_rows = 25)


,drug
isoosmoticdextrose,20.622194
bag,15.210765
vial,7.228978
polyethyleneglycol,7.047468
influenzavaccinequadrivalent,5.838108
neutraphos,5.117210
influenzavirusvaccine,5.024741
simethicone,3.390417
soln,3.070489
pneumococcalvacpolyvalent,2.874534


Lots of vaccines, let us fill those first

In [38]:
cat = list('ABCDGHJLMNPRSV')
vaccines_bool = (cat_pres.drug.str.contains('vaccine|polyvalent|measlesmumpsrubellavac',regex = True, na = False))
vaccines_bool *= cat_pres.A<0
cat_pres.loc[vaccines_bool,cat] = 0
cat_pres.loc[vaccines_bool,'J'] = 1

print('% of missing values:',nachopy.cat_check(cat_pres)*100)

% of missing values: 7.856632008480606


Let us have a look at the missing values and the drugs per drug type (from the prescriptions table)

In [39]:
print('% of missing values for MAIN drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='MAIN'])*100)
print('% of missing values for BASE drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='BASE'])*100)
print('% of missing values for ADDITIVE drugs',nachopy.cat_check(cat_pres[cat_pres.drug_type =='ADDITIVE'])*100)

base_drugs_bool = cat_pres.drug_type == 'BASE'
print('\n',cat_pres.drug[base_drugs_bool].drop_duplicates().values)

% of missing values for MAIN drugs 3.185890591466149
% of missing values for BASE drugs 25.785027717319707
% of missing values for ADDITIVE drugs 4.945122703169318

 ['sodiumchloride' 'lactatedringers' 'isoosmoticdextrose'
 'potassiumchlmeqmldns' 'dlr' 'lr' 'sw' 'ns' 'dns' 'dextrose' 'bag' 'vial'
 'sodiumchlorideminibagplus' 'isoosmoticsodiumchloride' 'dw'
 'isotonicsodiumchloride' 'syringesodiumchloride' 'soln'
 'potassiumchlmeqmlns' 'syringe' 'sterilewater' 'nsminibagplus'
 'syringechemo' 'mcgvial' 'yellowcaddcassette' 'dextroseexcelbag'
 'syringeisoosmoticdextrose' 'epiduralbag' 'amp' 'bluecaddcassette'
 'naclexcelviaflobag' 'dextroseglassbottle' 'potassiumchlmeqmldlr'
 'nssyringeintrapleural' 'syringesterilewater' 'syringelr'
 'insulinsyringeu' 'sodiumchlorideexcelbag' 'syringesw'
 'aminoacidswdextrose' 'aminoacidsdextrose' 'plasmalyte' 'nsepiduralbag'
 'nsepiduralbagnacl' 'dwexcelbag' 'clickchangetochooseabasesolution'
 'nssyringe' 'dwglassbottle' 'citratedextroseacdacrrt' 'prisma

Rows with drug_type "BASE" correspond with procedures (bag, vial...), solvents (sodium chloride, dextrose) and treatments/drugs (lipid/fat emulsions, travasol, amino acids...) apparently using IV therapy. Many of these procedures will not be very specific per hospital department (sodium chloride, bags, dextrose...), but frequency of procedures could be interesting so registering these events in our table could be useful if we use frequency encoding instead of one-hot encoding. Given that most of these events involve IV therapy and solvents are classed typically both as B (Blood products) and as V (Various) in the ATC system, these prescription events have been classified as such. 

In [40]:
base_drugs_bool = base_drugs_bool * cat_pres.A<0 
cat_pres.loc[base_drugs_bool,cat] = 0

cat_pres.loc[base_drugs_bool,'V'] = 1
cat_pres.loc[base_drugs_bool,'B'] = 1
cat_pres.A.value_counts(normalize = True)*100

print('% of missing values:',nachopy.cat_check(cat_pres)*100)

% of missing values: 4.237109596227292


We still have 4.24% of missing values in our atc1 columns. Let us run the recursive filler function again.


In [41]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 4.237109596227292
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 4.230159656297725
Loop #1, missing values % after filling with drug column: 4.1881714861804165
Loop #1, missing values % after filling with formulary_drug_cd column: 4.1881714861804165
Loop #1, missing values % after filling with ndc column: 4.18801310429125

Loop #1, missing values % after re-imploding df: 4.179573482857267
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 4.18801310429125
Loop #2, missing values % after filling with drug column: 4.18801310429125
Loop #2, missing values % after filling with formulary_drug_cd column: 4.18801310429125
Loop #2, missing values % after filling with ndc column: 4.18801310429125

Loop #2, missing values % after re-imploding df: 4.179573482857267


We can try and fill manually the drugs that occur most frequently.We can do another round of recursive filling

https://go.drugbank.com/drugs/DB09512

In [42]:
missing_drugs = cat_pres.drug[cat_pres.A<0].value_counts(normalize = True).head(30)
display(missing_drugs*100)
print(f'We will fill {round(missing_drugs.sum()*100,2)}% of the remaining missing drugs manually')
missing_drugsl = missing_drugs.index.values

polyethyleneglycol                      22.796416
neutraphos                              16.552620
simethicone                             10.966969
potassiumphosphate                       6.884517
phosphorus                               5.459197
sarnalotion                              4.291537
epoetinalfa                              4.125747
readicatbariumsulfatesuspension          3.894750
tiotropiumbromide                        3.433283
sodiumpolystyrenesulfonate               2.693563
vasopressin                              2.027762
ferricgluconate                          1.531711
tuberculinprotein                        1.226267
leucovorincalcium                        1.095588
gastroviewdiatrizoatemegluminesodium     1.058100
lithiumcarbonate                         0.842943
psylliumwafer                            0.766912
gammagardliquid                          0.764800
caphosol                                 0.606401
vitaminbcomplex                          0.601913


We will fill 95.78% of the remaining missing drugs manually


In [43]:
cat_pres.loc[cat_pres.drug.isin(missing_drugsl),cat] = 0

cat_pres.loc[cat_pres.drug == 'polyethyleneglycol','B'] = 1
cat_pres.loc[cat_pres.drug == 'neutraphos',['A','B']] = 1
cat_pres.loc[cat_pres.drug == 'simethicone','A'] = 1
cat_pres.loc[cat_pres.drug == 'potassiumphosphate','B'] = 1
cat_pres.loc[cat_pres.drug == 'phosphorus','B'] = 1
cat_pres.loc[cat_pres.drug == 'epoetinalfa','B'] = 1
cat_pres.loc[cat_pres.drug == 'sarnalotion', 'D'] = 1
cat_pres.loc[cat_pres.drug == 'readicatbariumsulfatesuspension',['A','V']] = 1 # Contrast agent
cat_pres.loc[cat_pres.drug == 'tiotropiumbromide','R'] = 1
cat_pres.loc[cat_pres.drug == 'sodiumpolystyrenesulfonate','V'] = 1
cat_pres.loc[cat_pres.drug == 'vasopressin','H'] = 1
cat_pres.loc[cat_pres.drug == 'fishoilmega','C'] = 1
cat_pres.loc[cat_pres.drug == 'ferricgluconate','B'] = 1
cat_pres.loc[cat_pres.drug == 'leucovorincalcium','V'] = 1
cat_pres.loc[cat_pres.drug == 'tuberculinprotein','V'] = 1
cat_pres.loc[cat_pres.drug == 'gastroviewdiatrizoatemegluminesodium',['A','V']] = 1 # Contrast agent
cat_pres.loc[cat_pres.drug == 'lithiumcarbonate','N'] = 1
cat_pres.loc[cat_pres.drug == 'gammagardliquid', 'J'] = 1
cat_pres.loc[cat_pres.drug == 'psylliumwafer', 'A'] = 1
cat_pres.loc[cat_pres.drug == 'caphosol', 'V'] = 1
cat_pres.loc[cat_pres.drug == 'vitaminbcomplex', 'B'] = 1
cat_pres.loc[cat_pres.drug == 'hydrocerin','D'] = 1
cat_pres.loc[cat_pres.drug == 'fosaprepitant','A'] = 1
cat_pres.loc[cat_pres.drug == 'psyllium', 'A'] = 1
cat_pres.loc[cat_pres.drug == 'levalbuterolneb','R'] = 1
cat_pres.loc[cat_pres.drug == 'aquaphorointment','D'] = 1
cat_pres.loc[cat_pres.drug == 'psylliumpowder','A'] = 1
cat_pres.loc[cat_pres.drug == 'protaminesulfate','V'] = 1
cat_pres.loc[cat_pres.drug == 'transplantselfmedicationprogram','V'] = 1
cat_pres.loc[cat_pres.drug == 'bismuthsubsalicylate','A'] = 1
cat_pres.loc[cat_pres.drug == 'xopenexneb','R'] = 1

In [44]:
nachopy.rec_cat_filler(cat_pres,['gsn','drug','formulary_drug_cd','ndc'])

% of missing values before loop: 2.1215980915005153
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 2.112882279436674
Loop #1, missing values % after filling with drug column: 2.112650363098966
Loop #1, missing values % after filling with formulary_drug_cd column: 2.111875423141257
Loop #1, missing values % after filling with ndc column: 2.103503808999587

Loop #1, missing values % after re-imploding df: 2.089236509446292
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 2.1017955471950027
Loop #2, missing values % after filling with drug column: 2.1017842342029196
Loop #2, missing values % after filling with formulary_drug_cd column: 2.1017842342029196
Loop #2, missing values % after filling with ndc column: 2.1017842342029196

Loop #2, missing values % after re-imploding df: 2.087733298360513
Dataframe exploded, starting for loop #3

Loop #3, missing 

In [45]:
catnas = cat_pres[cat_pres.A < 0]
catnas[['ndc','drug_type','drug','formulary_drug_cd','gsn']].isna().sum()/len(catnas)*100

ndc                   7.399508
drug_type            94.689407
drug                 96.884561
formulary_drug_cd    96.884561
gsn                  95.162184
dtype: float64

In [46]:
cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).iloc[50]*100

0.3180704508443245

In [47]:
nachopy.display_sbs([cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).iloc[:60]*100],\
                    titles = ['ndc value counts'], max_rows = 10)
            

missing_ndcs = cat_pres.ndc[cat_pres.A<0].value_counts(normalize = True).head(30)
print(f'We could fill {round(missing_ndcs.sum()*100,2)}% of the remaining missing ndcs manually by filling the first 30 ndcs')

,ndc
100030116,7.928287
105850865,7.021317
100030115,6.088819
105720810,4.999633
107700128,4.999340
353560215,2.331245
109390331,1.748507
109390803,1.675151
123330025,1.600035
109390021,1.284019


We could fill 54.06% of the remaining missing ndcs manually by filling the first 30 ndcs


The problem is that the missing values are more distributed between many ndcs, so filling manually 30 of them would just tackle 53.83% of the missing categories by ndc. Moreover, browsing ndc is not so easy, as they are specific per supplier and can be harder to find. Even more if they could not be found on the FDA website. We will leave these values unfilled for now. We can save the merged, filled prescriptions df to be used later.

In [61]:
prescriptions_v1 = prescriptions[['subject_id','hadm_id','starttime']].join(cat_pres)
prescriptions_v1.to_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_v1.pkl')

## Section 2 - Adding ATC lvl2

In [52]:
atc2 = pd.get_dummies(mapped_NDC[['ndc','atc2']],prefix = '',prefix_sep = '')
atc2 = atc2.groupby(by = 'ndc', as_index=False).any()*1 # Category will be 1 when there is at least one row for ndc with that category
atc2.drop_duplicates(inplace = True)
display(atc2)

,ndc,A01,A02,A03,A04,A05,A06,A07,A08,A09,...,R03,R05,R06,R07,S01,S02,S03,V03,V04,V06
0,20503,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,20604,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21009,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,21433,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21434,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10157,820280004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10158,861690020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
10159,876510760,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10160,894110545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [68]:
prescriptions['ndc'] = prescriptions.ndc.fillna(0) 
cat_pres_v2 = pd.merge(prescriptions[['drug_type','drug','formulary_drug_cd','gsn','ndc']],atc2,on = 'ndc',how = 'left')

cat_pres_v2

,drug_type,drug,formulary_drug_cd,gsn,ndc,A01,A02,A03,A04,A05,...,R03,R05,R06,R07,S01,S02,S03,V03,V04,V06
0,MAIN,Acetaminophen,APAP500,004490,9041988,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MAIN,Furosemide,FURO40,008209,510790073,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MAIN,Raltegravir,RALT400,063231,60227,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MAIN,Heparin,HEPA5I,006549,633230262,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17628923,NaN,fluticasone,NaN,18368,122800270,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17628924,NaN,glipizide,NaN,1776,118451101,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17628925,NaN,ibuprofen,NaN,8350,105440071,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17628926,NaN,metronidazole,NaN,9592,105440938,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
cat_pres_v2.to_pickle('G:/My Drive/Capstone-Data/pickles/cat_pres_v2.pkl')

In [4]:
cat_pres_v2['ndc'] = cat_pres_v2.ndc.astype('Int64')

In [5]:
# Converting columns to int8 to make it easier for the computer

i = 0
for col in cat_pres_v2.columns[5:]:
    cat_pres_v2[col] = cat_pres_v2[col].fillna(-1).astype('int8')
    i += 1
    print(f'{i}', end = '\r')
print()

86


In [ ]:
cat_pres_v2.to_pickle('G:/My Drive/Capstone-Data/pickles/cat_pres_v2.pkl') # Checkpoint

In [7]:
cat_pres_v2['ndc'] = cat_pres_v2.ndc.where(cat_pres_v2.ndc>0)

pd.options.mode.chained_assignment = None 

lvl2_cat = list(cat_pres_v2.columns[5:])
filler_cols = ['gsn','drug','formulary_drug_cd','ndc']

for first_col in range(0,73,12):
    last_col = first_col + 12 if first_col < 72 else len(lvl2_cat) - 1
    cat_cols = lvl2_cat[first_col:last_col]
    print(f'Columns from {cat_cols[0]} to {cat_cols[-1]}:')
    
    filling_df = cat_pres_v2[filler_cols + cat_cols]
    nachopy.rec_cat_filler(filling_df,filler_cols=filler_cols,cat = cat_cols)
    
    cat_pres_v2[cat_cols] = filling_df[cat_cols]

Columns from A01 to A12:
% of missing values before loop: 17.792244656056226
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 16.88708994716776
Loop #1, missing values % after filling with drug column: 9.639981562085502
Loop #1, missing values % after filling with formulary_drug_cd column: 8.22818802803744
Loop #1, missing values % after filling with ndc column: 7.705935061502232

Loop #1, missing values % after re-imploding df: 7.716561097759319
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 7.679909523214515
Loop #2, missing values % after filling with drug column: 6.841266107084596
Loop #2, missing values % after filling with formulary_drug_cd column: 6.748261999167476
Loop #2, missing values % after filling with ndc column: 6.623072428773261

Loop #2, missing values % after re-imploding df: 6.638259569725396
Dataframe exploded, starting for loop #

Loop #1, missing values % after filling with drug column: 9.639981562085502
Loop #1, missing values % after filling with formulary_drug_cd column: 8.22818802803744
Loop #1, missing values % after filling with ndc column: 7.705935061502232

Loop #1, missing values % after re-imploding df: 7.716561097759319
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 7.679909523214515
Loop #2, missing values % after filling with drug column: 6.841266107084596
Loop #2, missing values % after filling with formulary_drug_cd column: 6.748261999167476
Loop #2, missing values % after filling with ndc column: 6.623072428773261

Loop #2, missing values % after re-imploding df: 6.638259569725396
Dataframe exploded, starting for loop #3

Loop #3, missing values % after filling with gsn column: 6.622682130546385
Loop #3, missing values % after filling with drug column: 6.61449718077409
Loop #3, missing values % after filling with formulary_drug_cd column: 6.607

In [34]:
# Extra step just because initially I left some columns in loop
first_col = 72
last_col = first_col + 12 if first_col < 72 else len(lvl2_cat)
cat_cols = lvl2_cat[first_col:last_col]
print(f'Columns from {cat_cols[0]} to {cat_cols[-1]}:')

filling_df = cat_pres_v2[filler_cols + cat_cols]
nachopy.rec_cat_filler(filling_df,filler_cols=filler_cols,cat = cat_cols)

cat_pres_v2[cat_cols] = filling_df[cat_cols]

Columns from P02 to V06:
% of missing values before loop: 17.792244656056226
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 16.88708994716776
Loop #1, missing values % after filling with drug column: 9.639981562085502
Loop #1, missing values % after filling with formulary_drug_cd column: 8.22818802803744
Loop #1, missing values % after filling with ndc column: 7.705935061502232

Loop #1, missing values % after re-imploding df: 7.716561097759319
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 7.679909523214515
Loop #2, missing values % after filling with drug column: 6.841266107084596
Loop #2, missing values % after filling with formulary_drug_cd column: 6.748261999167476
Loop #2, missing values % after filling with ndc column: 6.623072428773261

Loop #2, missing values % after re-imploding df: 6.638259569725396
Dataframe exploded, starting for loop #

Let's look at missing drugs

In [104]:
missing_drugs = cat_pres_v2.drug[cat_pres_v2.A01<0].value_counts(normalize = True).head(30)
display(missing_drugs*100)
print(f'We could fill {round(missing_drugs.sum()*100,2)}% of the remaining missing drugs manually')
missing_drugsl = missing_drugs.index.values

Iso-Osmotic Dextrose                              22.047797
Bag                                               16.262278
Vial                                               7.728714
Influenza Vaccine Quadrivalent                     6.241694
Neutra-Phos                                        5.471135
Influenza Virus Vaccine                            5.372099
Soln                                               2.970112
Pneumococcal Vac Polyvalent                        2.823347
PNEUMOcoccal 23-valent polysaccharide vaccine      2.400853
Potassium Phosphate                                2.275466
Phosphorus                                         1.804370
Epoetin Alfa                                       1.365559
Syringe (0.9% Sodium Chloride)                     1.304828
Readi-Cat 2 (Barium Sulfate 2% Suspension)         1.287290
Tiotropium Bromide                                 1.137209
Sodium Polystyrene Sulfonate                       0.890798
Yellow CADD Cassette                    

We will fill 87.83% of the remaining missing drugs manually


We forgot to transform the drug column to homogenise caps, spaces, punctuation... We could repeat everything but life is short and the deadline is approaching. We can just apply the loop again and the result will be the same, although we will first fill vaccines and base drugs.  

In [111]:
# Transforming drugs column to make category-matching easier

nndrug = cat_pres_v2[cat_pres_v2.drug.isna()].index.values[0]

cat_pres_v2.loc[nndrug,['drug', 'formulary_drug_cd']] = ('Simvastatin','SIMV40')

cat_pres_v2.drug = cat_pres_v2.drug.str.lower()

clin_drug = cat_pres_v2.drug.str.contains('clindamyci')
cat_pres_v2.drug[clin_drug] = 'clindamycin'

cat_pres_v2['drug'] = cat_pres_v2.drug.str.replace('[^a-zA-Z]','', regex = True)

In [37]:
cat_pres_v2.to_pickle('../cat_pres_v2_filled1.pkl')

In [109]:
cat_pres_v2 = pd.read_pickle('../cat_pres_v2_filled1.pkl')

In [106]:
print('% of missing values for MAIN drugs',nachopy.cat_check(cat_pres_v2[cat_pres_v2.drug_type =='MAIN'])*100)
print('% of missing values for BASE drugs',nachopy.cat_check(cat_pres_v2[cat_pres_v2.drug_type =='BASE'])*100)
print('% of missing values for ADDITIVE drugs',nachopy.cat_check(cat_pres_v2[cat_pres_v2.drug_type =='ADDITIVE'])*100)

% of missing values for MAIN drugs 3.463973948225938
% of missing values for BASE drugs 25.34338522265587
% of missing values for ADDITIVE drugs 0.8180211287869446


Similar to previous version. Let's fill these and the vaccines

In [112]:
vaccines_bool = (cat_pres_v2.drug.str.contains('vaccine|polyvalent|measlesmumpsrubellavac',regex = True, na = False))
vaccines_bool *= cat_pres_v2.A01<0

cat_pres_v2.loc[vaccines_bool,lvl2_cat] = 0
cat_pres_v2.loc[vaccines_bool,'J07'] = 1

print('% of missing values after filling vaccines:',nachopy.cat_check(cat_pres_v2)*100)

% of missing values after filling vaccines: 5.37599336726544


In [113]:
base_drugs_bool = (cat_pres_v2.drug_type == 'BASE') * (cat_pres_v2.A01<0)

# Creating new column for non-theraepeutic prescriptions (solvents, bags...)
cat_pres_v2.loc[base_drugs_bool,lvl2_cat] = 0
cat_pres_v2['V07'] = 0
cat_pres_v2.loc[base_drugs_bool,'V07'] = 1
cat_pres_v2.loc[cat_pres_v2.A01<0,'V07'] = -1 # Labelling nulls as -1 for the same rows as other categories

lvl2_cat.append('V07') # Adding new column to category list

print('% of missing values after filling BASE drugs:',nachopy.cat_check(cat_pres_v2)*100)

% of missing values after filling BASE drugs: 1.8184656491875173


In [114]:
for first_col in range(0,73,12):
    last_col = first_col + 12 if first_col < 72 else len(lvl2_cat)
    cat_cols = lvl2_cat[first_col:last_col]
    print(f'Columns from {cat_cols[0]} to {cat_cols[-1]}:')
    
    filling_df = cat_pres_v2[filler_cols + cat_cols]
    nachopy.rec_cat_filler(filling_df,filler_cols=filler_cols,cat = cat_cols)
    
    cat_pres_v2[cat_cols] = filling_df[cat_cols]

Columns from A01 to A12:
% of missing values before loop: 1.8184656491875173
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 1.801475202854675
Loop #1, missing values % after filling with drug column: 1.70001463335526
Loop #1, missing values % after filling with formulary_drug_cd column: 1.6989908075717177
Loop #1, missing values % after filling with ndc column: 1.6750751423075416

Loop #1, missing values % after re-imploding df: 1.6764887802593558
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 1.6619011630264948
Loop #2, missing values % after filling with drug column: 1.6608716807469108
Loop #2, missing values % after filling with formulary_drug_cd column: 1.6583036315439934
Loop #2, missing values % after filling with ndc column: 1.653489953412533

Loop #2, missing values % after re-imploding df: 1.6548595581081276
Dataframe exploded, starting for

Loop #1, missing values % after filling with gsn column: 1.801475202854675
Loop #1, missing values % after filling with drug column: 1.70001463335526
Loop #1, missing values % after filling with formulary_drug_cd column: 1.6989908075717177
Loop #1, missing values % after filling with ndc column: 1.6750751423075416

Loop #1, missing values % after re-imploding df: 1.6764887802593558
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 1.6619011630264948
Loop #2, missing values % after filling with drug column: 1.6608716807469108
Loop #2, missing values % after filling with formulary_drug_cd column: 1.6583036315439934
Loop #2, missing values % after filling with ndc column: 1.653489953412533

Loop #2, missing values % after re-imploding df: 1.6548595581081276
Dataframe exploded, starting for loop #3

Loop #3, missing values % after filling with gsn column: 1.653456014436283
Loop #3, missing values % after filling with drug column: 1.651685531

Loop #1, missing values % after filling with formulary_drug_cd column: 1.6989908075717177
Loop #1, missing values % after filling with ndc column: 1.6750751423075416

Loop #1, missing values % after re-imploding df: 1.6764887802593558
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 1.6619011630264948
Loop #2, missing values % after filling with drug column: 1.6608716807469108
Loop #2, missing values % after filling with formulary_drug_cd column: 1.6583036315439934
Loop #2, missing values % after filling with ndc column: 1.653489953412533

Loop #2, missing values % after re-imploding df: 1.6548595581081276
Dataframe exploded, starting for loop #3

Loop #3, missing values % after filling with gsn column: 1.653456014436283
Loop #3, missing values % after filling with drug column: 1.6516855311752405
Loop #3, missing values % after filling with formulary_drug_cd column: 1.6516855311752405
Loop #3, missing values % after filling with ndc col

In [115]:
missing_drugs = cat_pres_v2.drug[cat_pres_v2.A01<0].value_counts(normalize = True).head(30)
display(missing_drugs*100)
print(f'We could fill {round(missing_drugs.sum()*100,2)}% of the remaining missing drugs manually')
missing_drugsl = missing_drugs.index.values

neutraphos                                 21.613379
potassiumphosphate                          8.986650
phosphorus                                  7.127842
epoetinalfa                                 5.463737
readicatbariumsulfatesuspension             5.083981
tiotropiumbromide                           4.692852
sodiumpolystyrenesulfonate                  3.803423
vasopressin                                 2.646923
ferricgluconate                             2.001130
bloodsugardiagnosticfreestylelitestrips     1.911877
tiotropiumbromidespirivawithhandihaler      1.619995
tuberculinprotein                           1.600697
leucovorincalcium                           1.527296
gastroviewdiatrizoatemegluminesodium        1.381183
bloodsugardiagnosticonetouchultratest       1.315708
gammagardliquid                             1.006940
spirivawithhandihaler                       0.929404
caphosol                                    0.796041
onetouchultratest                           0.

We could fill 78.71% of the remaining missing drugs manually


In [119]:
cat_pres_v2.to_pickle('G:/My Drive/Capstone-Data/pickles/cat_pres_v2_filling2.pkl') # Checkpoint

In [124]:
cat_pres_v2.loc[cat_pres_v2.drug.isin(missing_drugsl),lvl2_cat] = 0

cat_pres_v2.loc[cat_pres_v2.drug == 'neutraphos',['A12','B05']] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'potassiumphosphate',['A12','B05']] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'phosphorus','B05'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'epoetinalfa','B03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'sarnalotion', 'D07'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'readicatbariumsulfatesuspension','A16'] = 1 # Contrast agent, should be V08
cat_pres_v2.loc[cat_pres_v2.drug == 'tiotropiumbromide','R03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'sodiumpolystyrenesulfonate','V03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'vasopressin','H01'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'ferricgluconate','B03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'spirivawithhandihaler','R03'] = 1 # Diabetes drug
cat_pres_v2.loc[cat_pres_v2.drug == 'tuberculinprotein','V04'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'gastroviewdiatrizoatemegluminesodium','A16'] = 1 # Contrast agent
cat_pres_v2.loc[cat_pres_v2.drug == 'tiotropiumbromidespirivawithhandihaler','R03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'leucovorincalcium', 'V03'] = 1                           
cat_pres_v2.loc[cat_pres_v2.drug == 'gammagardliquid', 'J06'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'psylliumwafer', 'A06'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'vitaminbcomplex', 'A11'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'hydrocerin','D02'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'levalbuterolneb','R03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'protaminesulfate','V03'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'transplantselfmedicationprogram','V07'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'bismuthsubsalicylate','A07'] = 1
cat_pres_v2.loc[cat_pres_v2.drug == 'xopenexneb','R03'] = 1

# A few diabetes drug, we can fill all of those
diabetes_drugs = cat_pres_v2.drug.str.contains('bloodsugar|bloodglucose|onetouch')
cat_pres_v2.loc[diabetes_drugs,lvl2_cat] = 0
cat_pres_v2.loc[diabetes_drugs,'A10'] = 1
                
print('% of missing values after manual filling:',nachopy.cat_check(cat_pres_v2)*100)
print('Any NaN values created during fillin?',cat_pres_v2[cat_pres_v2.columns[5:]].isna().any().any())

% of missing values after manual filling: 0.3070294461466971


We can do a last round of recursive filling to see if we can fill more values

In [128]:
for first_col in range(0,73,12):
    last_col = first_col + 12 if first_col < 72 else len(lvl2_cat)
    cat_cols = lvl2_cat[first_col:last_col]
    print(f'Columns from {cat_cols[0]} to {cat_cols[-1]}:')
    
    filling_df = cat_pres_v2[filler_cols + cat_cols]
    nachopy.rec_cat_filler(filling_df,filler_cols=filler_cols,cat = cat_cols)
    
    cat_pres_v2[cat_cols] = filling_df[cat_cols]

Columns from A01 to A12:
% of missing values before loop: 0.3070294461466971
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 0.3043025175536628
Loop #1, missing values % after filling with drug column: 0.2932440677922
Loop #1, missing values % after filling with formulary_drug_cd column: 0.29245781484240807
Loop #1, missing values % after filling with ndc column: 0.26766539269177314

Loop #1, missing values % after re-imploding df: 0.26535930034997024
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 0.2662795511615643
Loop #2, missing values % after filling with drug column: 0.26595713088718914
Loop #2, missing values % after filling with formulary_drug_cd column: 0.26595713088718914
Loop #2, missing values % after filling with ndc column: 0.26499552656010544

Loop #2, missing values % after re-imploding df: 0.26268188286888455
Dataframe exploded, star

Loop #5, missing values % after filling with drug column: 0.23897564476842845
Loop #5, missing values % after filling with formulary_drug_cd column: 0.23897564476842845
Loop #5, missing values % after filling with ndc column: 0.2388851408317617

Loop #5, missing values % after re-imploding df: 0.23649764750301325
Dataframe exploded, starting for loop #6

Loop #6, missing values % after filling with gsn column: 0.2388851408317617
Loop #6, missing values % after filling with drug column: 0.23870978945447
Loop #6, missing values % after filling with formulary_drug_cd column: 0.23870978945447
Loop #6, missing values % after filling with ndc column: 0.23870978945447

Loop #6, missing values % after re-imploding df: 0.2363218001684504
Dataframe exploded, starting for loop #7

Loop #7, missing values % after filling with gsn column: 0.23870978945447
Loop #7, missing values % after filling with drug column: 0.23870978945447
Loop #7, missing values % after filling with formulary_drug_cd column:

Loop #2, missing values % after filling with ndc column: 0.26499552656010544

Loop #2, missing values % after re-imploding df: 0.26268188286888455
Dataframe exploded, starting for loop #3

Loop #3, missing values % after filling with gsn column: 0.26499552656010544
Loop #3, missing values % after filling with drug column: 0.26452038089260527
Loop #3, missing values % after filling with formulary_drug_cd column: 0.26452038089260527
Loop #3, missing values % after filling with ndc column: 0.2641866476261468

Loop #3, missing values % after re-imploding df: 0.2618707161320303
Dataframe exploded, starting for loop #4

Loop #4, missing values % after filling with gsn column: 0.2641866476261468
Loop #4, missing values % after filling with drug column: 0.25993861909885346
Loop #4, missing values % after filling with formulary_drug_cd column: 0.25993861909885346
Loop #4, missing values % after filling with ndc column: 0.23964311130134536

Loop #4, missing values % after re-imploding df: 0.2372

In [ ]:
filling_df = cat_pres_v2[filler_cols + cat_cols]
nachopy.rec_cat_filler(filling_df,filler_cols=filler_cols,cat = cat_cols)

cat_pres_v2[cat_cols] = filling_df[cat_cols]

Columns from A01 to A12:
% of missing values before loop: 0.3070294461466971
gsn column split, starting loop...
Dataframe exploded, starting for loop #1

Loop #1, missing values % after filling with gsn column: 0.3043025175536628
Loop #1, missing values % after filling with drug column: 0.2932440677922
Loop #1, missing values % after filling with formulary_drug_cd column: 0.29245781484240807
Loop #1, missing values % after filling with ndc column: 0.26766539269177314

Loop #1, missing values % after re-imploding df: 0.26535930034997024
Dataframe exploded, starting for loop #2

Loop #2, missing values % after filling with gsn column: 0.2662795511615643
Loop #2, missing values % after filling with drug column: 0.26595713088718914
Loop #2, missing values % after filling with formulary_drug_cd column: 0.26595713088718914
Loop #2, missing values % after filling with ndc column: 0.26499552656010544

Loop #2, missing values % after re-imploding df: 0.26268188286888455
Dataframe exploded, star

Loop #5, missing values % after filling with drug column: 0.23897564476842845
Loop #5, missing values % after filling with formulary_drug_cd column: 0.23897564476842845
Loop #5, missing values % after filling with ndc column: 0.2388851408317617

Loop #5, missing values % after re-imploding df: 0.23649764750301325
Dataframe exploded, starting for loop #6

Loop #6, missing values % after filling with gsn column: 0.2388851408317617
Loop #6, missing values % after filling with drug column: 0.23870978945447
Loop #6, missing values % after filling with formulary_drug_cd column: 0.23870978945447
Loop #6, missing values % after filling with ndc column: 0.23870978945447

Loop #6, missing values % after re-imploding df: 0.2363218001684504
Dataframe exploded, starting for loop #7

Loop #7, missing values % after filling with gsn column: 0.23870978945447
Loop #7, missing values % after filling with drug column: 0.23870978945447
Loop #7, missing values % after filling with formulary_drug_cd column:

Loop #2, missing values % after filling with ndc column: 0.26499552656010544

Loop #2, missing values % after re-imploding df: 0.26268188286888455
Dataframe exploded, starting for loop #3

Loop #3, missing values % after filling with gsn column: 0.26499552656010544
Loop #3, missing values % after filling with drug column: 0.26452038089260527
Loop #3, missing values % after filling with formulary_drug_cd column: 0.26452038089260527
Loop #3, missing values % after filling with ndc column: 0.2641866476261468

Loop #3, missing values % after re-imploding df: 0.2618707161320303
Dataframe exploded, starting for loop #4

Loop #4, missing values % after filling with gsn column: 0.2641866476261468
Loop #4, missing values % after filling with drug column: 0.25993861909885346
Loop #4, missing values % after filling with formulary_drug_cd column: 0.25993861909885346
Loop #4, missing values % after filling with ndc column: 0.23964311130134536

Loop #4, missing values % after re-imploding df: 0.2372

That is probably enough, let's join to prescriptions and save our progress

In [131]:
cat_pres_v2.to_pickle('G:/My Drive/Capstone-Data/pickles/cat_pres_v2_filling3.pkl') # Final checkpoint

prescriptions_v2 = pd.read_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_v1.pkl')
prescriptions_v2 = prescriptions_v2[['subject_id','hadm_id','starttime']].join(cat_pres_v2)

prescriptions_v2.to_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_v2.pkl')